# メモ書き
## MIXED PRECISION TRAINING

### 1 INTRODUCTION
- 最近のDNNモデルは徐々に巨大化しており、それに伴いより多くの計算やメモリーへのリソースが学習の際に必要となる。それらの必要となるリソースは、精度を低くすることにより、減らすことができる。
- 学習や推論等を行う速度は以下の3つによって決まる。
    1. 演算速度
    2. メモリ速度
    3. レイテンシ<br>
    最初の2つは扱う精度を低くすることにより、向上させることができる。<br>
        - 精度を低くすることにより(例:FP32→FP16)、同じ値をより少ないビット数でメモリに持たせることができる。
        - より少ない精度を使用することにより、演算しなければならないビット数も減らせることができる。例を挙げると、FP16を使用して最新のGPUで演算を行うと、FP32で演算した時と比べて、2倍～8倍のスループットを実現する。
- 今回提案する手法は、FP32の代わりにFP16を使用し、様々なNNモデルでaccuracyを保ちつつ、演算速度をあげることを検証する。
- FP32からFP16に代えても、モデルのaccuracyを保つために以下3つの提案する。
    1. FP32で重みの値を保持する
    2. 勾配が0になることを防ぐためのロススケーリング
    3. FP16とFP32の混合演算
    <br><span style="color: red; ">**※**</span>なお上3つのことを行う上で、モデルやハイパーパラメータを変更する必要はない
    
### 2 RELATED WORK
- 当論文で提案する手法は以下3つの点で今までの手法とは異なる。
    1. 全ての順伝播と逆伝播でFP16を用いて演算する
    2. FP16を使用する際もハイパーパラメータを変更する必要がない(レイヤーのノード数を変更する等)
    3. 当手法でFP16を用いて学習させたモデルは、FP32で学習させたモデルと比べて、同精度である

### 3 IMPLEMENTATION
#### 3.1 FP32での重みの保持
- FP32で重みを保持することが(精度を保つ上で)当たり前というわけではないが、以下2つの理由で多くのネットワークで必要する。
    1. 重みを更新する際に、更新する値、つまり重みの勾配 x 学習率、がFP16では0になり得る。そのため、重みの更新の際はFP32を使用することでこの問題を解決できる。
    2. 重みの更新値に対して重みの値がとても大きいとき。**これについては詳細を調査する必要あり**
- 重みをFP32で保持することが必要であることを示すために、Mandarin(中国の標準語)スピーチモデルを用いて検証したところ、混合演算を行った結果はFP32で演算した結果と同程度の精度を出すことができたが、FP16で重みの更新をしたところ、FP32で演算した結果の80%も精度を失った。
- 混合演算の際に追加で重みをFP32で保持することは、FP32で演算するときと比べて50%ほどメモリーの容量を多くとるが、総合的に言うと、圧倒的にメモリー使用量を削減できる。その理由は、学習時のメモリー使用量の大部分の原因はアクティベーションのところであり(より大きなバッチサイズを使用したり、逆伝播の演算の際にアクティベーションの値を使用するため、保持する必要がある)、その全てのアクティベーションがFP16で演算されるため、DNNを学習させるのに必要なメモリー量を大まかに言って約半分ほどに削減できる。
#### 3.2 ロススケーリング
- 下図はFP32を用いてSSDを学習させたときに、全てのレイヤーから得た活性化の勾配値のヒストグラムである。
- 下図から見てわかるように、FP16で扱うことができる範囲にほとんど値がなく、多くの値はFP16で扱うことができる最小値より小さい値である(その場合FP16では扱うことができないため、FP16を用いて演算を行った場合0になる)。
- そのため、大部分の活性化の勾配値をカバーするためにはスケールアップすることが必要であり(以下のケースでは8倍する)、そうすることによりFP32を用いた場合と同精度を出すことができる。
![loss_scaling](https://user-images.githubusercontent.com/37681936/71586989-64211a80-2b5f-11ea-8fb3-70b25a2bb496.png)
- 勾配値をFP16で扱える範囲にシフトするための一つの効果的な方法として、順伝播時に得たロス値をスケールアップすることである。そうすることにより、逆伝播時もチェインルールにより、全ての勾配値もスケールアップされた状態になる。ただし、スケールアップされた勾配値は、FP32で学習する時の更新幅と合わせるために、アップデート前に元に戻す(スケールダウン)する必要がある。
- ロススケーリングの値を選ぶ方法はいくつかあるが、一番簡単な方法は、定数を選ぶことだ。それは経験則を基に選んだり、統計情報を基に選べば良い。ただし、ロススケーリングの値と勾配値の絶対値が一番高い値を掛けた時にFP16の最大値(65,504)を超えてしまうと、オーバーフローになってまうため、注意が必要である。
#### 3.3 演算精度
- ニューラルネットワークの演算は大まかに以下3つのカテゴリーに分かれる。
    1. ベクトルのドット積
    2. リダクション(?)
    3. 要素ごとの演算
- ベクトルのドット積を行う際に、精度を保つために、途中計算で行う積の演算をFP32で蓄積しておく必要がある。そして、最終的にメモリーに書き込む前にFP16に変換する(NVIDIA Volta GPUsからはこういった混合演算さんを可能にするTensor Coresという機能が追加されている)。
- 全ての要素の和等の大きなリダクションを行うときはFP32で実行されるべきである。そういった演算は大体が統計情報を蓄積するバッチノーマライゼーションや、ソフトマックスレイヤーで行われる。私たちの実装では両方のレイヤーでFP16を用いてメモリーから読み込んだり、書き込んだりを行っており、演算はFP32で行っている。これらのレイヤーはmemory-bandwidth limited(?)であり、演算スピードによらないため、学習プロセスを遅くすることはない。
- 非線形変換や要素ごとの行列の積などの要素ごとの演算はmemory-bandwidth limited(?)である。演算精度はこれらの演算スピードに影響を与えないため、FP16でもFP32でも使用可能である。

### 4 RESULTS
- 今回の実験は様々なDNNモデルを用いて行った。それぞれのモデルで以下2つの方法で実験した。<br>
    **1. Baseline(FP32):** 活性化、重み、勾配をFP32で保持する。全ての演算もFP32を用いて行う。<br>
    **2. Mixed Presicion (MP、混合演算):** 活性化、重み、勾配をFP16で保持し、演算もFP16を用いて行う。FP32で保持する重みの値は更新時に使用される。ロススケーリングはいくつかのモデルで使用する。そしてFP16の演算では、畳み込み、全結合層、リカレント層の行列の積の演算を行う際に、(途中の)演算の蓄積の部分をFP32で行うTensor Coreを用いて実験する。<br>
- Baselineの検証はNVIDIAのMaxwell、またはPascalを用いて行う。
- Mixed Precisionの検証はNVIDIAのVolta 100を用いて行う。<br>
#### 4.1 ILSVRCにおいての畳み込み分類
- Alexnet, VGG-D, GoogLeNet, Inception v2, Inception v3, and pre-activation Resnet-50のモデルを用いて混合演算の実験を行った。すべてのモデルでハイパーパラメータを変えずにBaselineと同じようなTop-1の精度を出すことができた。
![cnn_classification_results](https://user-images.githubusercontent.com/37681936/71821752-4cedaa00-30d6-11ea-87f3-87a5ebc96a08.png)
- これらのネットワークではロススケーリングを使用する必要がなかった。
- 順伝播時と逆伝播時の全てのテンソルはFP16で保持され、マスターコピーの重みはFP32で更新された(3.1で説明したように)。<br>
#### 4.2 畳み込みでの検出
- 物体検出するために座標を予測する必要があるが、これは回帰によって予測される。
- 物体検出は検出されたそれぞれの物体がどのクラスに該当するかを予測する分類も行われる。
- 実験はFaster-RCNNとMultibox-SSDのモデルを使用して行った。そして両モデルでVGG16がバックボーンとして使用された。
- 検証するにあたって、Pascal VOC 2017のテストセットを用いてMean Average Precision(mAP)を算出した。
![object_detection_results](https://user-images.githubusercontent.com/37681936/71821754-4eb76d80-30d6-11ea-9bd0-ce9503a43ae5.png)
- 上のテーブルからわかるようにSSDをFP16を用いてロススケーリングなしで学習さえると発散した。FP16では扱いきれない小さな勾配値を0にすることによって、全体で見たときに望ましくない重みの更新が行われ、発散した。
- 3.2で説明したように、ロススケーリングを行うために8をかけることにより、FP16でも的確な勾配値を得ることができ、FP32で学習させた時と同精度の結果が出せる。<br>
#### 4.3 音声認識
- DeepSpeech 2のモデルを使って、英語とマンダリン(中国語の標準語)のデータセットで検証する。
- 英語のデータセットに使ったモデルは2層の2D Conv、3層のGPUセル使用のリカレント層、1層のrow conv、CTC Loss層からなる。パラメータ数は1億１千五百万。
- マンダリン用のモデルも同じようなアーキテクチャで、パラメータ数は2億１千五百万。
- 学習はbaseline時とFP16時のハイパーパラメータを変更せずに行う。
![speech_recognition_results](https://user-images.githubusercontent.com/37681936/71821760-4fe89a80-30d6-11ea-8107-8183eef97400.png)
- リカレント層を使用するモデルでも混合演算はFP32と同精度を出すことができた。
- 上のテーブルから混合演算の方が5%～10%精度が良いが、FP16を使用することが正則化の役割を果たしているのかもしれない。<br>
#### 4.4 機械翻訳
- GoogleがTensorFlowのチュートリアルで出している英語-フランス語翻訳のモデルを基に3つの異なるモデルを用いて検証した。
- 英語は10万語、フランス語は4万語の単語を使用。
- ネットワークはエンコーダ、デコーダ共にLSTM3層、または5層からなるモデルを使用。LSTMのセルの数は1024個。
- WMT15データセットを使用。
![NMT_results](https://user-images.githubusercontent.com/37681936/71821764-5119c780-30d6-11ea-818b-5416a029ea3e.png)
- 3つのモデルでそれぞれ精度が異なったが、ロススケーリングを用いた場合、混合演算はFP32と同精度の結果を出すことができた。ロススケーリングを用いなかった場合は、少し精度が低下した。
- 5層の場合でも同じような結果を得た。<br>
#### 4.5 言語モデル
- ここではbigLSTMと呼ばれるモデルで、10億語からなる英語のデータセットを使って、言語モデルを学習させる。このモデルは2層の8192個のLSTMセルと、1024次元に投射するエンベディング層からなる。
- 8千語のネガティブサンプルを用いて、ソフトマックスを計算する。
- バッチサイズは4つのGPUを使って1024に設定した。
- ロススケーリングなしで混合演算を行った場合は、300Kイテレーションを超えると値が発散し始めるため、128をかけてロススケーリングした結果、baselineと同精度を出すことができた。
![LM_results](https://user-images.githubusercontent.com/37681936/71988192-c8845e00-3272-11ea-98be-9aa933e0a797.png)<br>
#### 4.6 DCGAN
- test
***
## Training With Mixed Precision :: DEEP LEARNING SDK DOCUMENTATION
- Mixed Presicion Trainingを使用するためには、以下2つのステップが必要。<br>
    1. 計算上FP16で事が足りる箇所はFP16を適用する<br>
    2. 勾配の値が小さくなっても保持するためにロススケーリングを使用する<br>
- より小さな精度で演算を行うメリットは以下の2つ<br>
    1. **メモリー使用量を削減**<br>
    例を挙げると、普段使用する精度はFP32(32ビット必要)。対してFP16(16ビット必要)。すなわちFP32からFP16に変更することで16ビッド(32ビット-16ビット)も削減できる。そうすることにより、より大きなモデル、またはより大きなミニバッチで学習させることができる。<br>
    2. **学習時間、または推論時間の短縮**<br>
    上の例で挙げたように、FP16を使用することにより、FP32を使用する時と比べて、アクセスするバイト数を半分(16ビット)にできる。そのため、計算時間も短縮できる。NVIDIAのGPUでは演算にFP16を使用すると、FP32で計算する時と比べて、最大8倍のスループットを実現する。